In [44]:
import subprocess

In [81]:
def create_video_with_audio(images_dict, audio_dict, resolution, output_file):
    # Create video from images
    temp_video_file = "temp_video.mp4"
    sorted_images = images_dict.items()

    # Create a temporary file with the list of images and durations
    with open("images_list.txt", "w") as f:
        for image_path, duration in sorted_images:
            f.write(f"file '{image_path}'\n")
            f.write(f"duration {duration}\n")

    # Construct the ffmpeg command for video creation
    scale_filter = f"scale=w={resolution[0]}:h={resolution[1]}:force_original_aspect_ratio=1,pad={resolution[0]}:{resolution[1]}:(ow-iw)/2:(oh-ih)/2"
    ffmpeg_cmd = f"ffmpeg -f concat -i images_list.txt -vf '{scale_filter}' -r 30 -y '{temp_video_file}'"
    subprocess.run(ffmpeg_cmd, shell=True)

    # Add audio to the video
    sorted_audio = audio_dict.items()
    audio_input_str = ""
    audio_filter_str = ""
    for idx, (audio_path, start_time) in enumerate(sorted_audio, 1):
        audio_input_str += f"-i '{audio_path}' "
        audio_filter_str += f"[{idx}:a]adelay={int(start_time * 1000)}|{int(start_time * 1000)}[a{idx}];"

    # Concatenate audio streams
    audio_streams_str = "".join(f"[a{idx}]" for idx in range(1, len(sorted_audio) + 1))
    audio_filter_str += f"{audio_streams_str}amix=inputs={len(sorted_audio)}[a]"

    # Construct the ffmpeg command for audio addition
    ffmpeg_cmd = f"ffmpeg -i {temp_video_file} {audio_input_str}-filter_complex '{audio_filter_str}' -map 0:v -map '[a]' -c:v copy -y '{output_file}'"
    subprocess.run(ffmpeg_cmd, shell=True)

In [82]:
# Determine the video length
# Will add 4 seconds to the audio script length for now, as a naive approach
import wave
import contextlib

fname = "ted/script.wav"
with contextlib.closing(wave.open(fname, "r")) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
video_duration = int(duration) + 4
video_duration

162

In [83]:
scene_timecodes = open("ted/script_timecodes.txt", "r").readlines()
images_dict = {}
for idx, tc in enumerate(scene_timecodes):
    if idx + 1 < len(scene_timecodes):
        images_dict[f"ted/scene_{idx+1}.png"] = int(scene_timecodes[idx + 1]) - int(
            scene_timecodes[idx]
        )
    else:
        images_dict[f"ted/scene_{idx+1}.png"] = video_duration - int(
            scene_timecodes[idx]
        )

audio_dict = {"ted/script.wav": 2}
resolution = (1920, 1080)  # Output resolution in 1080p
output_file = "ted/output_video.mp4"

In [84]:
create_video_with_audio(images_dict, audio_dict, resolution, "ted/output_video.mp4")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab